In [96]:
import requests
import env
from requests.models import Request, Response
from env import Client_ID
import pandas as pd
from igdb.igdbapi_pb2 import GameResult
from typing import Dict, List, Optional, Union, cast
import acquire
import json
from igdb.wrapper import IGDBWrapper
import numpy as np
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split

### Connect to API and scrape the data needed
#### get the access token and 200 response 

In [ ]:
def get_api():   
    url = env.get_db_url()
    response = requests.post(url)
    data = response.json()
    access_token = data['access_token']
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *;'
    r = requests.post('https://api.igdb.com/v4/games', data=data, headers=headers)
    return access_token, r

In [2]:
access_token = acquire.connect_api()

In [3]:
tf = acquire.connect_to_games()

In [4]:
wrapper = acquire.run_wrapper()

In [5]:
game_library = acquire.get_game_library(wrapper)
game_library.head()

,id,category,created_at,external_games,first_release_date,genres,name,platforms,release_dates,similar_games,...,aggregated_rating_count,multiplayer_modes,remakes,dlcs,ports,expansions,expanded_games,remasters,forks,standalone_expansions
0,40104,0,1498435200,"[20127, 1988884]",5.364576e+08,[5],Dogou Souken,[52],[91007],"[12364, 27270, 43367, 103281, 103292, 103298, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,85031,0,1517392813,"[216805, 1969751, 2005770]",1.404346e+09,[9],City Mysteries,[37],[138333],"[10603, 19222, 25905, 41349, 86974, 87507, 885...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,99234,0,1524843905,"[603512, 1857496, 1917109]",1.527120e+09,"[31, 32]",Sword of the Black Stone,[6],[188385],"[25646, 29783, 68271, 80916, 96217, 105269, 10...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,148930,0,1621959514,"[2052375, 2057300]",NaN,"[13, 15, 31, 32]",Bok-Bok: A Chicken Dating Sim,NaN,NaN,"[25311, 26145, 28277, 33603, 51577, 65827, 791...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,139538,0,1602368467,[1966278],1.525910e+09,[32],sun machine,[82],[213594],"[11666, 13200, 13660, 17130, 25311, 28070, 348...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## scraping endpoints
scrape the end points of each category and add them to the game_librarys

### platform_families

In [7]:
def get_platform_families():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/platform_families' ,data=data ,  headers=headers)
    platform_families = wrapper.api_request('platform_families', 'fields *;limit 500;')
    x = json.loads(platform_families)
    platform_families =pd.DataFrame(x)
    return platform_families

In [8]:
platform_families = get_platform_families()
platform_families.head()

,id,name,slug,checksum
0,5,Nintendo,nintendo,398e5c87-f7db-6265-6da6-433441385208
1,4,Linux,linux,be8b89fa-823c-790a-1d5a-7e74315c1b27
2,2,Xbox,xbox,5bb9bcec-4aa7-45b8-129b-a6f88932942e
3,3,Sega,sega,bd1a9cc4-3632-33b0-ff4b-feab8cd15001
4,1,PlayStation,playstation,eadc0208-7972-9364-88b9-d2a9de75a998


### platform_version_release_dates

In [ ]:
def get_platform_version_release():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/platform_version_release_dates' ,data=data ,  headers=headers)
    platform_version_release_dates = wrapper.api_request('platform_version_release_dates', 'fields *;limit 500;')
    x = json.loads(platform_version_release_dates)
    platform_version_release_dates =pd.DataFrame(x)
    return platform_version_release_dates

In [ ]:
platform_version_release_dates = get_platform_version_release()
platform_version_release_dates

In [ ]:
# drop columns not needed
platform_version_release_dates = platform_version_release_dates.drop(columns=['category', 'date', 'm', 'y', 'region', 'updated_at', 'checksum', 'updated_at'])
platform_version_release_dates.head()


In [ ]:
df.head()

In [ ]:
version_dates_dict[9] = 'not_available'
version_dates_dict[384] = 'not_available'
version_dates_dict[386] = 'not_available'


In [ ]:
version_dates_list = platform_version_release_dates[['game' , 'human']].sort_values(by='game').reset_index(drop=True)
version_dates_dict = platform_version_release_dates.set_index('game').to_dict()['human']

In [ ]:
def test_funct3(random_list):
    if type(random_list) == list:
        return [version_dates_dict[i] for i in random_list]
    else:  
        return "Not available"

In [ ]:
platforms.set_index('Locality', inplace=True)
data.set_index('Locality', inplace=True)


In [ ]:
df['release_dates'] = df.release_dates.apply(test_funct3)
df.head()

In [ ]:
all_release_dates = pd.merge(release_dates, platform_version_release_dates, how = 'inner', on="human")

### platform_version_companies

In [ ]:
def get_platform_version_companies():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/platform_version_companies' ,data=data ,  headers=headers)
    platform_version_companies = wrapper.api_request('platform_version_companies', 'fields *;limit 500;')
    x = json.loads(platform_version_companies)
    platform_version_companies =pd.DataFrame(x)
    return platform_version_companies

In [ ]:
platform_version_companies=get_platform_version_companies()
platform_version_companies.head()

### platforms (use this)

In [9]:
def get_platforms():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/platforms' ,data=data ,  headers=headers)
    platforms = wrapper.api_request('platforms', 'fields *;limit 500;')
    x = json.loads(platforms)
    platforms =pd.DataFrame(x)
    return platforms

In [10]:
platform = get_platforms()
platform.shape

(197, 16)

In [ ]:
# see if there is a number in id that matches the big df
platform.loc[platform['id'] == 5]

In [ ]:
# looking at the dictionary
print(game_library.get(92, "Not found"))

In [11]:
platformlist = platform[['id' , 'name']].sort_values(by='id').reset_index(drop=True)


In [12]:
platform_dict = platformlist.set_index('id').to_dict()['name']

In [13]:
platform_dict[92] = 'not_available'

In [14]:
def test_funct(random_list):
    if type(random_list) == list:
        return [platform_dict[i] for i in random_list]
    else:  
        return  ["Not available"]

In [15]:
game_library['platforms'] = game_library.platforms.apply(test_funct)
game_library.head()

,id,category,created_at,external_games,first_release_date,genres,name,platforms,release_dates,similar_games,slug,summary,tags,updated_at,url,checksum,age_ratings,cover,involved_companies,screenshots,game_modes,player_perspectives,themes,websites,alternative_names,artworks,game_engines,keywords,status,videos,rating,rating_count,storyline,total_rating,total_rating_count,version_parent,version_title,hypes,collection,franchises,follows,bundles,parent_game,franchise,aggregated_rating,aggregated_rating_count,multiplayer_modes,remakes,dlcs,ports,expansions,expanded_games,remasters,forks,standalone_expansions
0,40104,0,1498435200,"[20127, 1988884]",5.364576e+08,[5],Dogou Souken,[Arcade],[91007],"[12364, 27270, 43367, 103281, 103292, 103298, ...",dogou-souken,An overhead-view shoot'em up game.,[268435461],1604620800,https://www.igdb.com/games/dogou-souken,5e7446ff-58d9-e744-f0d3-5e10241304c1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,85031,0,1517392813,"[216805, 1969751, 2005770]",1.404346e+09,[9],City Mysteries,[Nintendo 3DS],[138333],"[10603, 19222, 25905, 41349, 86974, 87507, 885...",city-mysteries,"Search buildings, streets, landmarks as you se...",[268435465],1641425677,https://www.igdb.com/games/city-mysteries,87b342e7-b912-ca8d-4ffb-f91f9e4f1870,[37944],61466.0,"[130531, 130532]","[155025, 155026, 155027]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,99234,0,1524843905,"[603512, 1857496, 1917109]",1.527120e+09,"[31, 32]",Sword of the Black Stone,[PC (Microsoft Windows)],[188385],"[25646, 29783, 68271, 80916, 96217, 105269, 10...",sword-of-the-black-stone,If your looking for action and adventure then ...,"[1, 268435487, 268435488]",1643857546,https://www.igdb.com/games/sword-of-the-black-...,0b575316-96dc-b056-86e0-c958c1c95efb,NaN,91582.0,"[155850, 155852]","[231528, 231529, 231530, 231531, 231532]",[1],[2],[1],[84617],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,148930,0,1621959514,"[2052375, 2057300]",NaN,"[13, 15, 31, 32]",Bok-Bok: A Chicken Dating Sim,[Not available],NaN,"[25311, 26145, 28277, 33603, 51577, 65827, 791...",bok-bok-a-chicken-dating-sim,Have you ever pondered what the romantic life ...,"[268435469, 268435471, 268435487, 268435488]",1643850601,https://www.igdb.com/games/bok-bok-a-chicken-d...,9585e273-2606-970f-3088-60261ea77068,NaN,169077.0,NaN,"[463145, 463148, 463149, 463151, 463152]",NaN,NaN,NaN,[192858],[63465],[33736],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,139538,0,1602368467,[1966278],1.525910e+09,[32],sun machine,[Web browser],[213594],"[11666, 13200, 13660, 17130, 25311, 28070, 348...",sun-machine,"a story of dark days, chronic fatigue, optimis...","[18, 268435488, 536896982]",1639609570,https://www.igdb.com/games/sun-machine,9986cb06-667c-bb20-d9cf-cb239d7020df,NaN,117024.0,[117090],"[403247, 403248]",[1],[3],[18],[156114],NaN,NaN,[837],[26070],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### release_dates

In [ ]:
def get_release_dates():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/release_dates' ,data=data ,  headers=headers)
    release_dates = wrapper.api_request('release_dates', 'fields *;limit 500;')
    x = json.loads(release_dates)
    release_dates =pd.DataFrame(x)
    return release_dates

In [ ]:
release_dates = get_release_dates()
release_dates.shape

In [ ]:
release_dates = release_dates.drop(columns=['category', 'date', 'm', 'region', 'updated_at', 'y', 'checksum', 'created_at', 'platform'])
release_dates.head()

In [ ]:
release_dates

In [ ]:
# add to dictionary
release_dict[91007] = "not-available"
release_dict[138333] = "not-available"
release_dict[188385] = "not-available"
release_dict[213594] = "not-available"
release_dict[77755] = "not-available"


In [ ]:
releaselist = release_dates[['game' , 'human']].sort_values(by='game').reset_index(drop=True)
release_dict = releaselist.set_index('game').to_dict()['human']

In [ ]:
release_dates.head()

In [ ]:
def test_funct2(random_list):
    if type(random_list) == list:
        return [release_dict[i] for i in random_list]
    else:  
        return  ["Not available"]

In [ ]:
release_dict

In [ ]:
df.release_dates.head()

In [ ]:
df['release_dates'] = pd.DataFrame(df.release_dates.apply(test_funct2))
df

In [ ]:
release_dict

In [ ]:
# add 0 and description to dictionary
release_dict["0"] = "not-available"

### platform_versions

In [ ]:
def get_platform_versions():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/platform_versions' ,data=data ,  headers=headers)
    platform_versions = wrapper.api_request('platform_versions', 'fields *;limit 500;')
    x = json.loads(platform_versions)
    platform_versions=pd.DataFrame(x)
    return platform_versions

In [ ]:
platform_versions = get_platform_versions()
platform_versions.shape

In [ ]:
platform_versions.head()

In [ ]:
df.columns

### player_perspectives (use this)

In [16]:
def get_player_perspectives():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/player_perspectives' ,data=data ,  headers=headers)
    player_perspectives = wrapper.api_request('player_perspectives', 'fields *;limit 500;')
    x = json.loads(player_perspectives)
    player_perspectives=pd.DataFrame(x)
    return player_perspectives

In [17]:
player_perspectives = get_player_perspectives()
player_perspectives.shape

(7, 7)

In [18]:
player_perspectives = player_perspectives.drop(columns=['created_at','updated_at', 'url', 'checksum'])
player_perspectives.head()


,id,name,slug
0,1,First person,first-person
1,2,Third person,third-person
2,3,Bird view / Isometric,bird-view-slash-isometric
3,5,Text,text
4,4,Side view,side-view


In [19]:
player_perspectives_list = player_perspectives[['id' , 'name']].sort_values(by='id').reset_index(drop=True)
player_perspectives_dict = player_perspectives.set_index('id').to_dict()['name']

In [20]:
def test_funct4(random_list):
    if type(random_list) == list:
        return [player_perspectives_dict[i] for i in random_list]
    else:  
        return  ["Not available"]

In [21]:
game_library['player_perspectives'] = pd.DataFrame(game_library.player_perspectives.apply(test_funct4))
game_library.head()

,id,category,created_at,external_games,first_release_date,genres,name,platforms,release_dates,similar_games,slug,summary,tags,updated_at,url,checksum,age_ratings,cover,involved_companies,screenshots,game_modes,player_perspectives,themes,websites,alternative_names,artworks,game_engines,keywords,status,videos,rating,rating_count,storyline,total_rating,total_rating_count,version_parent,version_title,hypes,collection,franchises,follows,bundles,parent_game,franchise,aggregated_rating,aggregated_rating_count,multiplayer_modes,remakes,dlcs,ports,expansions,expanded_games,remasters,forks,standalone_expansions
0,40104,0,1498435200,"[20127, 1988884]",5.364576e+08,[5],Dogou Souken,[Arcade],[91007],"[12364, 27270, 43367, 103281, 103292, 103298, ...",dogou-souken,An overhead-view shoot'em up game.,[268435461],1604620800,https://www.igdb.com/games/dogou-souken,5e7446ff-58d9-e744-f0d3-5e10241304c1,NaN,NaN,NaN,NaN,NaN,[Not available],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,85031,0,1517392813,"[216805, 1969751, 2005770]",1.404346e+09,[9],City Mysteries,[Nintendo 3DS],[138333],"[10603, 19222, 25905, 41349, 86974, 87507, 885...",city-mysteries,"Search buildings, streets, landmarks as you se...",[268435465],1641425677,https://www.igdb.com/games/city-mysteries,87b342e7-b912-ca8d-4ffb-f91f9e4f1870,[37944],61466.0,"[130531, 130532]","[155025, 155026, 155027]",NaN,[Not available],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,99234,0,1524843905,"[603512, 1857496, 1917109]",1.527120e+09,"[31, 32]",Sword of the Black Stone,[PC (Microsoft Windows)],[188385],"[25646, 29783, 68271, 80916, 96217, 105269, 10...",sword-of-the-black-stone,If your looking for action and adventure then ...,"[1, 268435487, 268435488]",1643857546,https://www.igdb.com/games/sword-of-the-black-...,0b575316-96dc-b056-86e0-c958c1c95efb,NaN,91582.0,"[155850, 155852]","[231528, 231529, 231530, 231531, 231532]",[1],[Third person],[1],[84617],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,148930,0,1621959514,"[2052375, 2057300]",NaN,"[13, 15, 31, 32]",Bok-Bok: A Chicken Dating Sim,[Not available],NaN,"[25311, 26145, 28277, 33603, 51577, 65827, 791...",bok-bok-a-chicken-dating-sim,Have you ever pondered what the romantic life ...,"[268435469, 268435471, 268435487, 268435488]",1643850601,https://www.igdb.com/games/bok-bok-a-chicken-d...,9585e273-2606-970f-3088-60261ea77068,NaN,169077.0,NaN,"[463145, 463148, 463149, 463151, 463152]",NaN,[Not available],NaN,[192858],[63465],[33736],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,139538,0,1602368467,[1966278],1.525910e+09,[32],sun machine,[Web browser],[213594],"[11666, 13200, 13660, 17130, 25311, 28070, 348...",sun-machine,"a story of dark days, chronic fatigue, optimis...","[18, 268435488, 536896982]",1639609570,https://www.igdb.com/games/sun-machine,9986cb06-667c-bb20-d9cf-cb239d7020df,NaN,117024.0,[117090],"[403247, 403248]",[1],[Bird view / Isometric],[18],[156114],NaN,NaN,[837],[26070],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### themes (use this)

In [22]:
def get_themes():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/themes' ,data=data ,  headers=headers)
    themes = wrapper.api_request('themes', 'fields *;limit 500;')
    x = json.loads(themes)
    themes=pd.DataFrame(x)
    return themes

In [23]:
themes = get_themes()
themes.shape

(22, 7)

In [24]:
themes_list = themes[['id' , 'name']].sort_values(by='id').reset_index(drop=True)
themes_dict = themes.set_index('id').to_dict()['name']

In [25]:
def test_funct5(random_list):
    if type(random_list) == list:
        return [themes_dict[i] for i in random_list]
    else:  
        return  ["Not available"]

In [26]:
game_library['themes'] = pd.DataFrame(game_library.themes.apply(test_funct5))
game_library.head()

,id,category,created_at,external_games,first_release_date,genres,name,platforms,release_dates,similar_games,slug,summary,tags,updated_at,url,checksum,age_ratings,cover,involved_companies,screenshots,game_modes,player_perspectives,themes,websites,alternative_names,artworks,game_engines,keywords,status,videos,rating,rating_count,storyline,total_rating,total_rating_count,version_parent,version_title,hypes,collection,franchises,follows,bundles,parent_game,franchise,aggregated_rating,aggregated_rating_count,multiplayer_modes,remakes,dlcs,ports,expansions,expanded_games,remasters,forks,standalone_expansions
0,40104,0,1498435200,"[20127, 1988884]",5.364576e+08,[5],Dogou Souken,[Arcade],[91007],"[12364, 27270, 43367, 103281, 103292, 103298, ...",dogou-souken,An overhead-view shoot'em up game.,[268435461],1604620800,https://www.igdb.com/games/dogou-souken,5e7446ff-58d9-e744-f0d3-5e10241304c1,NaN,NaN,NaN,NaN,NaN,[Not available],[Not available],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,85031,0,1517392813,"[216805, 1969751, 2005770]",1.404346e+09,[9],City Mysteries,[Nintendo 3DS],[138333],"[10603, 19222, 25905, 41349, 86974, 87507, 885...",city-mysteries,"Search buildings, streets, landmarks as you se...",[268435465],1641425677,https://www.igdb.com/games/city-mysteries,87b342e7-b912-ca8d-4ffb-f91f9e4f1870,[37944],61466.0,"[130531, 130532]","[155025, 155026, 155027]",NaN,[Not available],[Not available],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,99234,0,1524843905,"[603512, 1857496, 1917109]",1.527120e+09,"[31, 32]",Sword of the Black Stone,[PC (Microsoft Windows)],[188385],"[25646, 29783, 68271, 80916, 96217, 105269, 10...",sword-of-the-black-stone,If your looking for action and adventure then ...,"[1, 268435487, 268435488]",1643857546,https://www.igdb.com/games/sword-of-the-black-...,0b575316-96dc-b056-86e0-c958c1c95efb,NaN,91582.0,"[155850, 155852]","[231528, 231529, 231530, 231531, 231532]",[1],[Third person],[Action],[84617],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,148930,0,1621959514,"[2052375, 2057300]",NaN,"[13, 15, 31, 32]",Bok-Bok: A Chicken Dating Sim,[Not available],NaN,"[25311, 26145, 28277, 33603, 51577, 65827, 791...",bok-bok-a-chicken-dating-sim,Have you ever pondered what the romantic life ...,"[268435469, 268435471, 268435487, 268435488]",1643850601,https://www.igdb.com/games/bok-bok-a-chicken-d...,9585e273-2606-970f-3088-60261ea77068,NaN,169077.0,NaN,"[463145, 463148, 463149, 463151, 463152]",NaN,[Not available],[Not available],[192858],[63465],[33736],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,139538,0,1602368467,[1966278],1.525910e+09,[32],sun machine,[Web browser],[213594],"[11666, 13200, 13660, 17130, 25311, 28070, 348...",sun-machine,"a story of dark days, chronic fatigue, optimis...","[18, 268435488, 536896982]",1639609570,https://www.igdb.com/games/sun-machine,9986cb06-667c-bb20-d9cf-cb239d7020df,NaN,117024.0,[117090],"[403247, 403248]",[1],[Bird view / Isometric],[Science fiction],[156114],NaN,NaN,[837],[26070],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Genres (use this)

In [27]:
# function that puts response list object into a dataframe for each page
def get_genres(wrapper):
    genres = pd.DataFrame()
    for i in range (0, 409):
        genre = wrapper.api_request('genres', 'fields *; limit 500;')
        y = json.loads(genre)
        results_df =pd.DataFrame(y)
        genres = pd.concat([genres, results_df])
    return genres

In [28]:
genres = get_genres(wrapper)
genres.head()

,id,created_at,name,slug,updated_at,url,checksum
0,4,1297555200,Fighting,fighting,1323216000,https://www.igdb.com/genres/fighting,2ccc6572-bdde-6ed4-8843-25447ea40782
1,5,1297555200,Shooter,shooter,1323216000,https://www.igdb.com/genres/shooter,bb15fd3f-0f46-e5f3-2b40-d046cf9bd2ef
2,7,1297555200,Music,music,1323216000,https://www.igdb.com/genres/music,22d44a0d-89c7-580f-eef2-e06f178fdd47
3,8,1297555200,Platform,platform,1323216000,https://www.igdb.com/genres/platform,697fc5a4-b96f-a803-288a-498bd5dd1de1
4,9,1297555200,Puzzle,puzzle,1323216000,https://www.igdb.com/genres/puzzle,616de9c3-8a00-0232-9df9-00014cfac51b


In [29]:
genreslist = genres[['id' , 'slug']].sort_values(by='id').reset_index(drop=True)
genres_dict = genreslist.set_index('id').to_dict()['slug']

In [30]:
def test_functg(random_list):
    if type(random_list) == list:
        return [genres_dict[i] for i in random_list]
    else:  
        return  ["Not available"] 

In [31]:
game_library['genres'] = pd.DataFrame(game_library.genres.apply(test_functg))
game_library.head()

,id,category,created_at,external_games,first_release_date,genres,name,platforms,release_dates,similar_games,slug,summary,tags,updated_at,url,checksum,age_ratings,cover,involved_companies,screenshots,game_modes,player_perspectives,themes,websites,alternative_names,artworks,game_engines,keywords,status,videos,rating,rating_count,storyline,total_rating,total_rating_count,version_parent,version_title,hypes,collection,franchises,follows,bundles,parent_game,franchise,aggregated_rating,aggregated_rating_count,multiplayer_modes,remakes,dlcs,ports,expansions,expanded_games,remasters,forks,standalone_expansions
0,40104,0,1498435200,"[20127, 1988884]",5.364576e+08,[shooter],Dogou Souken,[Arcade],[91007],"[12364, 27270, 43367, 103281, 103292, 103298, ...",dogou-souken,An overhead-view shoot'em up game.,[268435461],1604620800,https://www.igdb.com/games/dogou-souken,5e7446ff-58d9-e744-f0d3-5e10241304c1,NaN,NaN,NaN,NaN,NaN,[Not available],[Not available],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,85031,0,1517392813,"[216805, 1969751, 2005770]",1.404346e+09,[puzzle],City Mysteries,[Nintendo 3DS],[138333],"[10603, 19222, 25905, 41349, 86974, 87507, 885...",city-mysteries,"Search buildings, streets, landmarks as you se...",[268435465],1641425677,https://www.igdb.com/games/city-mysteries,87b342e7-b912-ca8d-4ffb-f91f9e4f1870,[37944],61466.0,"[130531, 130532]","[155025, 155026, 155027]",NaN,[Not available],[Not available],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,99234,0,1524843905,"[603512, 1857496, 1917109]",1.527120e+09,"[adventure, indie]",Sword of the Black Stone,[PC (Microsoft Windows)],[188385],"[25646, 29783, 68271, 80916, 96217, 105269, 10...",sword-of-the-black-stone,If your looking for action and adventure then ...,"[1, 268435487, 268435488]",1643857546,https://www.igdb.com/games/sword-of-the-black-...,0b575316-96dc-b056-86e0-c958c1c95efb,NaN,91582.0,"[155850, 155852]","[231528, 231529, 231530, 231531, 231532]",[1],[Third person],[Action],[84617],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,148930,0,1621959514,"[2052375, 2057300]",NaN,"[simulator, strategy, adventure, indie]",Bok-Bok: A Chicken Dating Sim,[Not available],NaN,"[25311, 26145, 28277, 33603, 51577, 65827, 791...",bok-bok-a-chicken-dating-sim,Have you ever pondered what the romantic life ...,"[268435469, 268435471, 268435487, 268435488]",1643850601,https://www.igdb.com/games/bok-bok-a-chicken-d...,9585e273-2606-970f-3088-60261ea77068,NaN,169077.0,NaN,"[463145, 463148, 463149, 463151, 463152]",NaN,[Not available],[Not available],[192858],[63465],[33736],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,139538,0,1602368467,[1966278],1.525910e+09,[indie],sun machine,[Web browser],[213594],"[11666, 13200, 13660, 17130, 25311, 28070, 348...",sun-machine,"a story of dark days, chronic fatigue, optimis...","[18, 268435488, 536896982]",1639609570,https://www.igdb.com/games/sun-machine,9986cb06-667c-bb20-d9cf-cb239d7020df,NaN,117024.0,[117090],"[403247, 403248]",[1],[Bird view / Isometric],[Science fiction],[156114],NaN,NaN,[837],[26070],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Game modes (use this)

In [32]:
def get_game_modes():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; limit 500;'
    r = requests.post('https://api.igdb.com/v4/game_modes' ,data=data ,  headers=headers)
    game_modes = wrapper.api_request('game_modes', 'fields *;limit 500;')
    x = json.loads(game_modes)
    game_modes =pd.DataFrame(x)
    return game_modes

In [33]:
game_mode = get_game_modes()

In [34]:
modelist = game_mode[['id' , 'slug']].sort_values(by='id').reset_index(drop=True)
mode_dict = modelist.set_index('id').to_dict()['slug']

In [35]:
def test_functm(random_list):
    if type(random_list) == list:
        return [mode_dict[i] for i in random_list]
    else:  
        return  ["Not available"]

In [36]:
game_library['game_modes'] = pd.DataFrame(game_library.game_modes.apply(test_functm))
game_library.head()

,id,category,created_at,external_games,first_release_date,genres,name,platforms,release_dates,similar_games,slug,summary,tags,updated_at,url,checksum,age_ratings,cover,involved_companies,screenshots,game_modes,player_perspectives,themes,websites,alternative_names,artworks,game_engines,keywords,status,videos,rating,rating_count,storyline,total_rating,total_rating_count,version_parent,version_title,hypes,collection,franchises,follows,bundles,parent_game,franchise,aggregated_rating,aggregated_rating_count,multiplayer_modes,remakes,dlcs,ports,expansions,expanded_games,remasters,forks,standalone_expansions
0,40104,0,1498435200,"[20127, 1988884]",5.364576e+08,[shooter],Dogou Souken,[Arcade],[91007],"[12364, 27270, 43367, 103281, 103292, 103298, ...",dogou-souken,An overhead-view shoot'em up game.,[268435461],1604620800,https://www.igdb.com/games/dogou-souken,5e7446ff-58d9-e744-f0d3-5e10241304c1,NaN,NaN,NaN,NaN,[Not available],[Not available],[Not available],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,85031,0,1517392813,"[216805, 1969751, 2005770]",1.404346e+09,[puzzle],City Mysteries,[Nintendo 3DS],[138333],"[10603, 19222, 25905, 41349, 86974, 87507, 885...",city-mysteries,"Search buildings, streets, landmarks as you se...",[268435465],1641425677,https://www.igdb.com/games/city-mysteries,87b342e7-b912-ca8d-4ffb-f91f9e4f1870,[37944],61466.0,"[130531, 130532]","[155025, 155026, 155027]",[Not available],[Not available],[Not available],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,99234,0,1524843905,"[603512, 1857496, 1917109]",1.527120e+09,"[adventure, indie]",Sword of the Black Stone,[PC (Microsoft Windows)],[188385],"[25646, 29783, 68271, 80916, 96217, 105269, 10...",sword-of-the-black-stone,If your looking for action and adventure then ...,"[1, 268435487, 268435488]",1643857546,https://www.igdb.com/games/sword-of-the-black-...,0b575316-96dc-b056-86e0-c958c1c95efb,NaN,91582.0,"[155850, 155852]","[231528, 231529, 231530, 231531, 231532]",[single-player],[Third person],[Action],[84617],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,148930,0,1621959514,"[2052375, 2057300]",NaN,"[simulator, strategy, adventure, indie]",Bok-Bok: A Chicken Dating Sim,[Not available],NaN,"[25311, 26145, 28277, 33603, 51577, 65827, 791...",bok-bok-a-chicken-dating-sim,Have you ever pondered what the romantic life ...,"[268435469, 268435471, 268435487, 268435488]",1643850601,https://www.igdb.com/games/bok-bok-a-chicken-d...,9585e273-2606-970f-3088-60261ea77068,NaN,169077.0,NaN,"[463145, 463148, 463149, 463151, 463152]",[Not available],[Not available],[Not available],[192858],[63465],[33736],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,139538,0,1602368467,[1966278],1.525910e+09,[indie],sun machine,[Web browser],[213594],"[11666, 13200, 13660, 17130, 25311, 28070, 348...",sun-machine,"a story of dark days, chronic fatigue, optimis...","[18, 268435488, 536896982]",1639609570,https://www.igdb.com/games/sun-machine,9986cb06-667c-bb20-d9cf-cb239d7020df,NaN,117024.0,[117090],"[403247, 403248]",[single-player],[Bird view / Isometric],[Science fiction],[156114],NaN,NaN,[837],[26070],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Game version features (use this)

In [37]:
def get_game_version_features(wrapper):
    game_version_features = pd.DataFrame()
    for i in range (0, 409):
        a_ratings = wrapper.api_request('game_version_features', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(a_ratings)
        results_df =pd.DataFrame(y)
        game_version_features = pd.concat([game_version_features, results_df])
    return game_version_features

In [38]:
game_version_features = get_game_version_features(wrapper)
game_version_features.head()

,id,category,description,position,title,values,checksum
0,337,0.0,Sharpens your aim with rifles and pistols. Alw...,1,DLC: The Sharpshooter Detective Suit,"[1385, 1386, 1387, 1388]",a10d3ebf-ae3a-f0cc-fc68-116b1545a9ac
1,347,0.0,NaN,0,"25 cm (10"") figurine of an Alb Mage","[1419, 1420]",2a42989b-4936-75bc-6331-0ce79abb225a
2,352,0.0,NaN,5,A making of DVD,"[1429, 1430]",bb46d55f-7145-0b5c-2f69-cea9dd2570d6
3,375,0.0,Double sided print of the in-game island,2,Collectible map,"[1519, 1520, 1521, 1522, 1523]",8a9ea6d0-b5f1-ece2-cd5d-61636d639ad0
4,382,0.0,"Exclusive to TESCO in the UK, it included thre...",9,Combat Strike Pack,"[1554, 1555, 1556, 1557, 1558]",263031ce-acb2-69a2-ab66-240dbaad4f4d


In [39]:
game_vf_df = game_version_features[['id','description', 'title']]
game_vf_df.head()

,id,description,title
0,337,Sharpens your aim with rifles and pistols. Alw...,DLC: The Sharpshooter Detective Suit
1,347,NaN,"25 cm (10"") figurine of an Alb Mage"
2,352,NaN,A making of DVD
3,375,Double sided print of the in-game island,Collectible map
4,382,"Exclusive to TESCO in the UK, it included thre...",Combat Strike Pack


In [40]:
game_vf_df = game_vf_df.rename(columns={'description': 'version_description', 'title': 'title_version'})

## Collections

In [41]:
game_library['collection'] = game_library['collection'].fillna(0)
game_library['collection'] = game_library['collection'].astype(int)

In [ ]:
game_library.collection.value_counts()

In [ ]:
def get_collections(wrapper):
    collections = pd.DataFrame()
    for i in range (0, 409):
        a_ratings = wrapper.api_request('collections', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(a_ratings)
        results_df =pd.DataFrame(y)
        collections = pd.concat([collections, results_df])
    return collections

In [ ]:
collections = get_collections(wrapper)
collections

In [ ]:
collectionslist = collections[['id' , 'slug']].sort_values(by='id').reset_index(drop=True)
collections_dict = collectionslist.set_index('id').to_dict()['slug']

In [ ]:
game_library.collection.info()

In [ ]:
def test_funct4(random_list):
    if type(random_list) == list:
        return [collections_dict[i] for i in random_list]
    else:  
        return 'Not available'

In [ ]:
game_library['collection'] = pd.DataFrame(game_library.collection.apply(test_funct4))
game_library.tail(10)

## Game version feature values (use this)

In [42]:
def vf_values(wrapper):
    vf_values= pd.DataFrame()
    for i in range (0, 409):
        gvf_values = wrapper.api_request('game_version_feature_values', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(gvf_values)
        results_df =pd.DataFrame(y)
        vf_values = pd.concat([vf_values, results_df])
    return vf_values

In [43]:
gamevfv_df = vf_values(wrapper)
gamevfv_df.head()

,id,game,game_feature,included_feature,checksum,note
0,1581,52617,391,0.0,1db62035-9135-3d01-25a5-9c194d8fbd8f,NaN
1,1590,52617,394,0.0,e2e097d5-c88d-d6ea-48cf-1f5288f6bbe2,NaN
2,591,39045,144,0.0,6ee07864-0f1b-d41f-e2a5-e7dc2731d9a0,NaN
3,1987,83886,503,2.0,612d4e1b-a3b2-60e4-680a-064c0837f71e,NaN
4,1600,19333,399,0.0,ba6dcc29-45c7-6412-ebce-da6731910fb9,NaN


In [44]:
gamevfv_df = gamevfv_df[['game','game_feature']]
gamevfv_df.head()

,game,game_feature
0,52617,391
1,52617,394
2,39045,144
3,83886,503
4,19333,399


In [45]:
gamevfv_df = gamevfv_df.rename(columns={"game_feature": "id"})

In [46]:
gamevfv_df = gamevfv_df.set_index('id')
gamevfv_df.head()

,game
id,
391,52617
394,52617
144,39045
503,83886
399,19333


### Merging game feature version and game feature version values (use this)

In [47]:
results_df = pd.merge(gamevfv_df,  game_vf_df , how = 'left', on="id")
results_df.head()

,id,game,version_description,title_version
0,391,52617,"Delve into the mystery of the Shining Lady, Ca...",Field Report with Removable Coin
1,394,52617,Show the world you've joined the siege with th...,Siege of Dragonspear Amulet
2,144,39045,NaN,Bayek’s Eagle Skull Amulet
3,503,83886,Doomsday Prepper Pack includes the prepper out...,Doomsday Prepper Pack
4,399,19333,Prospect Mask with a display base,Prospect Mask


In [48]:
results_df = results_df.rename(columns={'id': 'second','game': 'id' })
results_df = results_df.set_index('id')
results_df.head()

,second,version_description,title_version
id,,,
52617,391,"Delve into the mystery of the Shining Lady, Ca...",Field Report with Removable Coin
52617,394,Show the world you've joined the siege with th...,Siege of Dragonspear Amulet
39045,144,NaN,Bayek’s Eagle Skull Amulet
83886,503,Doomsday Prepper Pack includes the prepper out...,Doomsday Prepper Pack
19333,399,Prospect Mask with a display base,Prospect Mask


In [49]:
#merging the to the main dataframe
game_library = pd.merge(game_library, results_df, how = 'left', on="id")
game_library.head()

,id,category,created_at,external_games,first_release_date,genres,name,platforms,release_dates,similar_games,slug,summary,tags,updated_at,url,checksum,age_ratings,cover,involved_companies,screenshots,game_modes,player_perspectives,themes,websites,alternative_names,artworks,game_engines,keywords,status,videos,rating,rating_count,storyline,total_rating,total_rating_count,version_parent,version_title,hypes,collection,franchises,follows,bundles,parent_game,franchise,aggregated_rating,aggregated_rating_count,multiplayer_modes,remakes,dlcs,ports,expansions,expanded_games,remasters,forks,standalone_expansions,second,version_description,title_version
0,40104,0,1498435200,"[20127, 1988884]",5.364576e+08,[shooter],Dogou Souken,[Arcade],[91007],"[12364, 27270, 43367, 103281, 103292, 103298, ...",dogou-souken,An overhead-view shoot'em up game.,[268435461],1604620800,https://www.igdb.com/games/dogou-souken,5e7446ff-58d9-e744-f0d3-5e10241304c1,NaN,NaN,NaN,NaN,[Not available],[Not available],[Not available],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,85031,0,1517392813,"[216805, 1969751, 2005770]",1.404346e+09,[puzzle],City Mysteries,[Nintendo 3DS],[138333],"[10603, 19222, 25905, 41349, 86974, 87507, 885...",city-mysteries,"Search buildings, streets, landmarks as you se...",[268435465],1641425677,https://www.igdb.com/games/city-mysteries,87b342e7-b912-ca8d-4ffb-f91f9e4f1870,[37944],61466.0,"[130531, 130532]","[155025, 155026, 155027]",[Not available],[Not available],[Not available],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,99234,0,1524843905,"[603512, 1857496, 1917109]",1.527120e+09,"[adventure, indie]",Sword of the Black Stone,[PC (Microsoft Windows)],[188385],"[25646, 29783, 68271, 80916, 96217, 105269, 10...",sword-of-the-black-stone,If your looking for action and adventure then ...,"[1, 268435487, 268435488]",1643857546,https://www.igdb.com/games/sword-of-the-black-...,0b575316-96dc-b056-86e0-c958c1c95efb,NaN,91582.0,"[155850, 155852]","[231528, 231529, 231530, 231531, 231532]",[single-player],[Third person],[Action],[84617],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,148930,0,1621959514,"[2052375, 2057300]",NaN,"[simulator, strategy, adventure, indie]",Bok-Bok: A Chicken Dating Sim,[Not available],NaN,"[25311, 26145, 28277, 33603, 51577, 65827, 791...",bok-bok-a-chicken-dating-sim,Have you ever pondered what the romantic life ...,"[268435469, 268435471, 268435487, 268435488]",1643850601,https://www.igdb.com/games/bok-bok-a-chicken-d...,9585e273-2606-970f-3088-60261ea77068,NaN,169077.0,NaN,"[463145, 463148, 463149, 463151, 463152]",[Not available],[Not available],[Not available],[192858],[63465],[33736],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,139538,0,1602368467,[1966278],1.525910e+09,[indie],sun machine,[Web browser],[213594],"[11666, 13200, 13660, 17130, 25311, 28070, 348...",sun-machine,"a story of dark days, chronic fatigue, optimis...","[18, 268435488, 536896982]",1639609570,https://www.igdb.com/games/sun-machine,9986cb06-667c-bb20-d9cf-cb239d7020df,NaN,117024.0,[117090],"[403247, 403248]",[single-player],[Bird view / Isometric],[Science fiction],[156114],NaN,NaN,[837],[26070],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Multi player mode (use this)

In [50]:
headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
data = 'fields *; limit 500;'
r = requests.post('https://api.igdb.com/v4/multiplayer_modes' ,data=data ,  headers=headers)
multiplayer_modes = wrapper.api_request('multiplayer_modes', 'fields *;limit 500;')
x = json.loads(multiplayer_modes)
multiplayer_modes =pd.DataFrame(x)

In [51]:
def multi_play(wrapper):
    multi_play= pd.DataFrame()
    for i in range (0, 409):
        multi_values = wrapper.api_request('multiplayer_modes', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(multi_values)
        results_df =pd.DataFrame(y)
        multi_play = pd.concat([multi_play, multiplayer_modes])
    return multi_play

In [52]:
multi_df = multi_play(wrapper)

In [53]:
def test_functg(random_list):
    if type(random_list) == list:
        return [genres_dict[i] for i in random_list]
    else:  
        return  ["Not available"] 

In [54]:
multi_df = multi_df.rename(columns={'game': 'id', 'id': 'second_id'})

In [55]:
multi_df = multi_df.set_index('id')

In [56]:
#marging this together with the main df
game_library = pd.merge(game_library, multi_df, how = 'left', on="id")
game_library.head()

,id,category,created_at,external_games,first_release_date,genres,name,platforms,release_dates,similar_games,slug,summary,tags,updated_at,url,checksum_x,age_ratings,cover,involved_companies,screenshots,game_modes,player_perspectives,themes,websites,alternative_names,artworks,game_engines,keywords,status,videos,rating,rating_count,storyline,total_rating,total_rating_count,version_parent,version_title,hypes,collection,franchises,follows,bundles,parent_game,franchise,aggregated_rating,aggregated_rating_count,multiplayer_modes,remakes,dlcs,ports,expansions,expanded_games,remasters,forks,standalone_expansions,second,version_description,title_version,second_id,campaigncoop,dropin,lancoop,offlinecoop,offlinemax,onlinecoop,platform,splitscreen,checksum_y,offlinecoopmax,onlinecoopmax,onlinemax
0,40104,0,1498435200,"[20127, 1988884]",5.364576e+08,[shooter],Dogou Souken,[Arcade],[91007],"[12364, 27270, 43367, 103281, 103292, 103298, ...",dogou-souken,An overhead-view shoot'em up game.,[268435461],1604620800,https://www.igdb.com/games/dogou-souken,5e7446ff-58d9-e744-f0d3-5e10241304c1,NaN,NaN,NaN,NaN,[Not available],[Not available],[Not available],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,85031,0,1517392813,"[216805, 1969751, 2005770]",1.404346e+09,[puzzle],City Mysteries,[Nintendo 3DS],[138333],"[10603, 19222, 25905, 41349, 86974, 87507, 885...",city-mysteries,"Search buildings, streets, landmarks as you se...",[268435465],1641425677,https://www.igdb.com/games/city-mysteries,87b342e7-b912-ca8d-4ffb-f91f9e4f1870,[37944],61466.0,"[130531, 130532]","[155025, 155026, 155027]",[Not available],[Not available],[Not available],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,99234,0,1524843905,"[603512, 1857496, 1917109]",1.527120e+09,"[adventure, indie]",Sword of the Black Stone,[PC (Microsoft Windows)],[188385],"[25646, 29783, 68271, 80916, 96217, 105269, 10...",sword-of-the-black-stone,If your looking for action and adventure then ...,"[1, 268435487, 268435488]",1643857546,https://www.igdb.com/games/sword-of-the-black-...,0b575316-96dc-b056-86e0-c958c1c95efb,NaN,91582.0,"[155850, 155852]","[231528, 231529, 231530, 231531, 231532]",[single-player],[Third person],[Action],[84617],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,148930,0,1621959514,"[2052375, 2057300]",NaN,"[simulator, strategy, adventure, indie]",Bok-Bok: A Chicken Dating Sim,[Not available],NaN,"[25311, 26145, 28277, 33603, 51577, 65827, 791...",bok-bok-a-chicken-dating-sim,Have you ever pondered what the romantic life ...,"[268435469, 268435471, 268435487, 268435488]",1643850601,https://www.igdb.com/games/bok-bok-a-chicken-d...,9585e273-2606-970f-3088-60261ea77068,NaN,169077.0,NaN,"[463145, 463148, 463149, 463151, 463152]",[Not available],[Not available],[Not available],[192858],[63465],[33736],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,139538,0,1602368467,[1966278],1.525910e+09,[indie],sun machine,[Web browser],[213594],"[11666, 13200, 13660, 17130, 25311, 28070, 348...",sun-machine,"a story of dark days, chronic fatigue, optimis...","[18, 268435488, 536896982]",1639609570,https://www.igdb.com/games/sun-machine,9986cb06-667c-bb20-d9cf-cb239d7020df,NaN,117024.0,[117090],"[403247, 403248]",[single-player],[Bird view / Isometric],[Science fiction],[156114],NaN,NaN,[837],[26070],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [57]:
game_library['campaigncoop'].fillna(False, inplace = True)
game_library['dropin'].fillna(False, inplace = True)
game_library['lancoop'].fillna(False, inplace = True)
game_library['offlinecoop'].fillna(False, inplace = True)
game_library['offlinemax'].fillna(0, inplace = True)
game_library['onlinecoop'].fillna(False, inplace = True)
game_library['splitscreen'].fillna(False, inplace = True)
game_library['offlinecoopmax'].fillna(0, inplace = True)
game_library['onlinecoopmax'].fillna(0, inplace = True)
game_library['onlinemax'].fillna(0, inplace = True)

### loop through genres list

In [58]:
# list of genres to add
items = ['point-and-click', 'fighting', 'shooter', 'music', 'platform', 'puzzle', 'racing', 'real-time-strategy-rts', 'role-playing-rpg', 'simulator', 'sport', 'strategy', 'turn-based-strategy-tbs', 'tactical', 'hack-and-slash-beat-em-up', 'quiz-trivia', 'pinball', 'adventure', 'indie', 'arcade', 'visual-novel', 'card-and-board-game', 'moba']
# function to loop through column list and check for genre
def my_list(column, word):
    if word in column:
        return 1
    else:
        return 0

In [59]:
for item in items:
    game_library[item] = game_library['genres'].apply(my_list, word=item)

In [60]:
game_library.head()

,id,category,created_at,external_games,first_release_date,genres,name,platforms,release_dates,similar_games,slug,summary,tags,updated_at,url,checksum_x,age_ratings,cover,involved_companies,screenshots,game_modes,player_perspectives,themes,websites,alternative_names,artworks,game_engines,keywords,status,videos,rating,rating_count,storyline,total_rating,total_rating_count,version_parent,version_title,hypes,collection,franchises,follows,bundles,parent_game,franchise,aggregated_rating,aggregated_rating_count,multiplayer_modes,remakes,dlcs,ports,expansions,expanded_games,remasters,forks,standalone_expansions,second,version_description,title_version,second_id,campaigncoop,dropin,lancoop,offlinecoop,offlinemax,onlinecoop,platform,splitscreen,checksum_y,offlinecoopmax,onlinecoopmax,onlinemax,point-and-click,fighting,shooter,music,puzzle,racing,real-time-strategy-rts,role-playing-rpg,simulator,sport,strategy,turn-based-strategy-tbs,tactical,hack-and-slash-beat-em-up,quiz-trivia,pinball,adventure,indie,arcade,visual-novel,card-and-board-game,moba
0,40104,0,1498435200,"[20127, 1988884]",5.364576e+08,[shooter],Dogou Souken,[Arcade],[91007],"[12364, 27270, 43367, 103281, 103292, 103298, ...",dogou-souken,An overhead-view shoot'em up game.,[268435461],1604620800,https://www.igdb.com/games/dogou-souken,5e7446ff-58d9-e744-f0d3-5e10241304c1,NaN,NaN,NaN,NaN,[Not available],[Not available],[Not available],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,0.0,False,0,False,NaN,0.0,0.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,85031,0,1517392813,"[216805, 1969751, 2005770]",1.404346e+09,[puzzle],City Mysteries,[Nintendo 3DS],[138333],"[10603, 19222, 25905, 41349, 86974, 87507, 885...",city-mysteries,"Search buildings, streets, landmarks as you se...",[268435465],1641425677,https://www.igdb.com/games/city-mysteries,87b342e7-b912-ca8d-4ffb-f91f9e4f1870,[37944],61466.0,"[130531, 130532]","[155025, 155026, 155027]",[Not available],[Not available],[Not available],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,0.0,False,0,False,NaN,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,99234,0,1524843905,"[603512, 1857496, 1917109]",1.527120e+09,"[adventure, indie]",Sword of the Black Stone,[PC (Microsoft Windows)],[188385],"[25646, 29783, 68271, 80916, 96217, 105269, 10...",sword-of-the-black-stone,If your looking for action and adventure then ...,"[1, 268435487, 268435488]",1643857546,https://www.igdb.com/games/sword-of-the-black-...,0b575316-96dc-b056-86e0-c958c1c95efb,NaN,91582.0,"[155850, 155852]","[231528, 231529, 231530, 231531, 231532]",[single-player],[Third person],[Action],[84617],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,0.0,False,0,False,NaN,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0
3,148930,0,1621959514,"[2052375, 2057300]",NaN,"[simulator, strategy, adventure, indie]",Bok-Bok: A Chicken Dating Sim,[Not available],NaN,"[25311, 26145, 28277, 33603, 51577, 65827, 791...",bok-bok-a-chicken-dating-sim,Have you ever pondered what the romantic life ...,"[268435469, 268435471, 268435487, 268435488]",1643850601,https://www.igdb.com/games/bok-bok-a-chicken-d...,9585e273-2606-970f-3088-60261ea77068,NaN,169077.0,NaN,"[463145, 463148, 463149, 463151, 463152]",[Not available],[Not available],[Not available],[192858],[63465],[33736],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,0.0,False,0,False,NaN,0.0,0.0,0.0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,1,0,0,0,0
4,139538,0,1602368467,[1966278],1.525910e+09,[indie],sun machine,[Web browser],[213594],"[11666, 13200, 13660, 17130, 2531

### Adding platforms to game_library

In [ ]:
len(platform_dict)

In [61]:
# list of platformd to add
platforms = ['Linux', 'Nintendo 64', 'Wii', 'PC (Microsoft Windows)', 'PlayStation', 'PlayStation 2', 'PlayStation 3', 'Xbox', 'Xbox 360', 'DOS', 'Mac', 'Nintendo Entertainment System', 'Super Nintendo Entertainment System', 'Nintendo DS', 'Nintendo GameCube', 'Game Boy Color', 'Game Boy Advance', 'Sega Mega Drive/Genesis','Sega Saturn', 'Game Boy', 'Android', 'Nintendo 3DS','PlayStation Portable', 'iOS', 'Wii U', 'PlayStation Vita', 'Virtual Console (Nintendo)', 'PlayStation 4', 'Xbox One', 'Family Computer Disk System', 'Arcade', 'Windows Phone', 'Apple II', 'Web browser', 'Odyssey', 'Commodore 16', 'New Nintendo 3DS']
 

In [62]:
# function to loop through column list and check for genre
def my_list(column, word):
    if word in column:
        return 1
    else:
        return 0

In [63]:
for platform in platforms:
    game_library[platform] = game_library['platforms'].apply(my_list, word=platform)

### Adding themes to game library 

In [64]:
themes = ['Thriller', 'Science fiction', 'Action', 'Horror', 'Survival', 'Fantasy', 'Historical', 'Stealth', 'Comedy', 'Business', 'Drama', 'Non-fiction', 'Kids', 'Sandbox', 'Open world', 'Warfare', 'Educational', 'Mystery', 'Party', 'Romance', 'Erotic']

def my_list_themes(column, word):
    if word in column:
        return 1
    else:
        return 0


In [65]:
for theme in themes:
    game_library[theme] = game_library['themes'].apply(my_list_themes, word=theme)

In [86]:
game_library.head()

,id,category,created_at,name,slug,rating,dlcs,campaigncoop,dropin,lancoop,offlinecoop,offlinemax,onlinecoop,platform,splitscreen,onlinemax,point-and-click,fighting,shooter,music,puzzle,racing,real-time-strategy-rts,role-playing-rpg,simulator,sport,strategy,turn-based-strategy-tbs,tactical,hack-and-slash-beat-em-up,quiz-trivia,pinball,adventure,indie,arcade,visual-novel,card-and-board-game,moba,Linux,Nintendo 64,Wii,PC (Microsoft Windows),PlayStation,PlayStation 2,PlayStation 3,Xbox,Xbox 360,DOS,Mac,Nintendo Entertainment System,Super Nintendo Entertainment System,Nintendo DS,Nintendo GameCube,Game Boy Color,Game Boy Advance,Sega Mega Drive/Genesis,Sega Saturn,Game Boy,Android,Nintendo 3DS,PlayStation Portable,iOS,Wii U,PlayStation Vita,Virtual Console (Nintendo),PlayStation 4,Xbox One,Family Computer Disk System,Arcade,Windows Phone,Apple II,Web browser,Odyssey,Commodore 16,New Nintendo 3DS,Thriller,Science fiction,Action,Horror,Survival,Fantasy,Historical,Stealth,Comedy,Business,Drama,Non-fiction,Kids,Sandbox,Open world,Warfare,Educational,Mystery,Party,Romance,Erotic,First person,Third person,Bird view / Isometric,Text,Side view,Virtual Reality,AuditoryDrama,single-player,multiplayer,co-operative,split-screen,massively-multiplayer-online-mmo,battle-royale,has_dlcs
0,40104,0,1498435200,Dogou Souken,dogou-souken,NaN,0,False,False,False,False,0.0,False,0,False,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,85031,0,1517392813,City Mysteries,city-mysteries,NaN,0,False,False,False,False,0.0,False,0,False,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,99234,0,1524843905,Sword of the Black Stone,sword-of-the-black-stone,NaN,0,False,False,False,False,0.0,False,0,False,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,148930,0,1621959514,Bok-Bok: A Chicken Dating Sim,bok-bok-a-chicken-dating-sim,NaN,0,False,False,False,False,0.0,False,0,False,0.0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,139538,0,1602368467,sun machine,sun-machine,NaN,0,False,False,False,False,0.0,False,0,False,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


### Add player perspective 

In [ ]:
player_perspectives_dict

In [67]:
players = ['First person', 'Third person', 'Bird view / Isometric', 'Text', 'Side view', 'Virtual Reality', 'Auditory'  'Drama']

def my_list_player(column, word):
    if word in column:
        return 1
    else:
        return 0


In [68]:
for player in players:
    game_library[player] = game_library['player_perspectives'].apply(my_list_player, word=player)

In [ ]:
game_library.head()

In [ ]:
game_library.shape

### Add game modes

In [ ]:
mode_dict

In [69]:
modes = ['single-player', 'multiplayer', 'co-operative', 'split-screen', 'massively-multiplayer-online-mmo', 'battle-royale']

def my_list(column, word):
    if word in column:
        return 1
    else:
        return 0

In [70]:
for mode in modes:
    game_library[mode] = game_library['player_perspectives'].apply(my_list, word=mode)

/var/folders/nr/4dgzb75x0x5b5kl5610rvf340000gn/T/ipykernel_88956/1687251994.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  game_library[mode] = game_library['player_perspectives'].apply(my_list, word=mode)
/var/folders/nr/4dgzb75x0x5b5kl5610rvf340000gn/T/ipykernel_88956/1687251994.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  game_library[mode] = game_library['player_perspectives'].apply(my_list, word=mode)
/var/folders/nr/4dgzb75x0x5b5kl5610rvf340000gn/T/ipykernel_88956/1687251994.py:2: PerformanceWarning: DataFrame 

In [ ]:
game_library.head()

In [ ]:
game_library.shape

### turn dlc column into boolean column

In [71]:
game_library['dlcs'] = game_library['dlcs'].fillna(0)
game_library.head()

,id,category,created_at,external_games,first_release_date,genres,name,platforms,release_dates,similar_games,slug,summary,tags,updated_at,url,checksum_x,age_ratings,cover,involved_companies,screenshots,game_modes,player_perspectives,themes,websites,alternative_names,artworks,game_engines,keywords,status,videos,rating,rating_count,storyline,total_rating,total_rating_count,version_parent,version_title,hypes,collection,franchises,follows,bundles,parent_game,franchise,aggregated_rating,aggregated_rating_count,multiplayer_modes,remakes,dlcs,ports,expansions,expanded_games,remasters,forks,standalone_expansions,second,version_description,title_version,second_id,campaigncoop,dropin,lancoop,offlinecoop,offlinemax,onlinecoop,platform,splitscreen,checksum_y,offlinecoopmax,onlinecoopmax,onlinemax,point-and-click,fighting,shooter,music,puzzle,racing,real-time-strategy-rts,role-playing-rpg,simulator,sport,strategy,turn-based-strategy-tbs,tactical,hack-and-slash-beat-em-up,quiz-trivia,pinball,adventure,indie,arcade,visual-novel,card-and-board-game,moba,Linux,Nintendo 64,Wii,PC (Microsoft Windows),PlayStation,PlayStation 2,PlayStation 3,Xbox,Xbox 360,DOS,Mac,Nintendo Entertainment System,Super Nintendo Entertainment System,Nintendo DS,Nintendo GameCube,Game Boy Color,Game Boy Advance,Sega Mega Drive/Genesis,Sega Saturn,Game Boy,Android,Nintendo 3DS,PlayStation Portable,iOS,Wii U,PlayStation Vita,Virtual Console (Nintendo),PlayStation 4,Xbox One,Family Computer Disk System,Arcade,Windows Phone,Apple II,Web browser,Odyssey,Commodore 16,New Nintendo 3DS,Thriller,Science fiction,Action,Horror,Survival,Fantasy,Historical,Stealth,Comedy,Business,Drama,Non-fiction,Kids,Sandbox,Open world,Warfare,Educational,Mystery,Party,Romance,Erotic,First person,Third person,Bird view / Isometric,Text,Side view,Virtual Reality,AuditoryDrama,single-player,multiplayer,co-operative,split-screen,massively-multiplayer-online-mmo,battle-royale
0,40104,0,1498435200,"[20127, 1988884]",5.364576e+08,[shooter],Dogou Souken,[Arcade],[91007],"[12364, 27270, 43367, 103281, 103292, 103298, ...",dogou-souken,An overhead-view shoot'em up game.,[268435461],1604620800,https://www.igdb.com/games/dogou-souken,5e7446ff-58d9-e744-f0d3-5e10241304c1,NaN,NaN,NaN,NaN,[Not available],[Not available],[Not available],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,0.0,False,0,False,NaN,0.0,0.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,85031,0,1517392813,"[216805, 1969751, 2005770]",1.404346e+09,[puzzle],City Mysteries,[Nintendo 3DS],[138333],"[10603, 19222, 25905, 41349, 86974, 87507, 885...",city-mysteries,"Search buildings, streets, landmarks as you se...",[268435465],1641425677,https://www.igdb.com/games/city-mysteries,87b342e7-b912-ca8d-4ffb-f91f9e4f1870,[37944],61466.0,"[130531, 130532]","[155025, 155026, 155027]",[Not available],[Not available],[Not available],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,0.0,False,0,False,NaN,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,99234,0,1524843905,"[603512, 1857496, 1917109]",1.527120e+09,"[adventure, indie]",Sword of the Black Stone,[PC (Microsoft Windows)],[188385],"[25646, 29783, 68271, 80916, 96217, 105269, 10...",sword-of-the-black-stone,If your looking for action and adventure then ...,"[1, 268435487, 268435488]",1643857546,https://www.igdb.com/games/sword-of-the-black-...,0b575316-96dc-b056-86e0-c958c1c95efb,NaN,91582.0,"[155850, 155852]","[231528, 231529, 231530, 231531, 231532]",[single-player],[Third person

In [72]:
def dlcs_col(df):
    game_library['has_dlcs'] = np.where(game_library.dlcs != 0, 1, 0)  
    return df

In [73]:
game_library = dlcs_col(game_library)
game_library.head()

/var/folders/nr/4dgzb75x0x5b5kl5610rvf340000gn/T/ipykernel_88956/3009727860.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  game_library['has_dlcs'] = np.where(game_library.dlcs != 0, 1, 0)


,id,category,created_at,external_games,first_release_date,genres,name,platforms,release_dates,similar_games,slug,summary,tags,updated_at,url,checksum_x,age_ratings,cover,involved_companies,screenshots,game_modes,player_perspectives,themes,websites,alternative_names,artworks,game_engines,keywords,status,videos,rating,rating_count,storyline,total_rating,total_rating_count,version_parent,version_title,hypes,collection,franchises,follows,bundles,parent_game,franchise,aggregated_rating,aggregated_rating_count,multiplayer_modes,remakes,dlcs,ports,expansions,expanded_games,remasters,forks,standalone_expansions,second,version_description,title_version,second_id,campaigncoop,dropin,lancoop,offlinecoop,offlinemax,onlinecoop,platform,splitscreen,checksum_y,offlinecoopmax,onlinecoopmax,onlinemax,point-and-click,fighting,shooter,music,puzzle,racing,real-time-strategy-rts,role-playing-rpg,simulator,sport,strategy,turn-based-strategy-tbs,tactical,hack-and-slash-beat-em-up,quiz-trivia,pinball,adventure,indie,arcade,visual-novel,card-and-board-game,moba,Linux,Nintendo 64,Wii,PC (Microsoft Windows),PlayStation,PlayStation 2,PlayStation 3,Xbox,Xbox 360,DOS,Mac,Nintendo Entertainment System,Super Nintendo Entertainment System,Nintendo DS,Nintendo GameCube,Game Boy Color,Game Boy Advance,Sega Mega Drive/Genesis,Sega Saturn,Game Boy,Android,Nintendo 3DS,PlayStation Portable,iOS,Wii U,PlayStation Vita,Virtual Console (Nintendo),PlayStation 4,Xbox One,Family Computer Disk System,Arcade,Windows Phone,Apple II,Web browser,Odyssey,Commodore 16,New Nintendo 3DS,Thriller,Science fiction,Action,Horror,Survival,Fantasy,Historical,Stealth,Comedy,Business,Drama,Non-fiction,Kids,Sandbox,Open world,Warfare,Educational,Mystery,Party,Romance,Erotic,First person,Third person,Bird view / Isometric,Text,Side view,Virtual Reality,AuditoryDrama,single-player,multiplayer,co-operative,split-screen,massively-multiplayer-online-mmo,battle-royale,has_dlcs
0,40104,0,1498435200,"[20127, 1988884]",5.364576e+08,[shooter],Dogou Souken,[Arcade],[91007],"[12364, 27270, 43367, 103281, 103292, 103298, ...",dogou-souken,An overhead-view shoot'em up game.,[268435461],1604620800,https://www.igdb.com/games/dogou-souken,5e7446ff-58d9-e744-f0d3-5e10241304c1,NaN,NaN,NaN,NaN,[Not available],[Not available],[Not available],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,0.0,False,0,False,NaN,0.0,0.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,85031,0,1517392813,"[216805, 1969751, 2005770]",1.404346e+09,[puzzle],City Mysteries,[Nintendo 3DS],[138333],"[10603, 19222, 25905, 41349, 86974, 87507, 885...",city-mysteries,"Search buildings, streets, landmarks as you se...",[268435465],1641425677,https://www.igdb.com/games/city-mysteries,87b342e7-b912-ca8d-4ffb-f91f9e4f1870,[37944],61466.0,"[130531, 130532]","[155025, 155026, 155027]",[Not available],[Not available],[Not available],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,0.0,False,0,False,NaN,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,99234,0,1524843905,"[603512, 1857496, 1917109]",1.527120e+09,"[adventure, indie]",Sword of the Black Stone,[PC (Microsoft Windows)],[188385],"[25646, 29783, 68271, 80916, 96217, 105269, 10...",sword-of-the-black-stone,If your looking for action and adventure then ...,"[1, 268435487, 268435488]",1643857546,https://www.igdb.com/games/sword-of-the-black-...,0b575316-96dc-b056-86e0-c958c1c95efb,NaN,91582.0,"[155850, 155852]","[231528, 231529, 231530, 231531, 231532]",[single-player],

#### Delete columns we dont need

In [74]:
game_library = game_library.drop(columns=['first_release_date', 'websites', 'collection', 'external_games', 'updated_at', 'artworks', 'genres', 'platforms', 'similar_games', 'tags', 'release_dates', 'cover','screenshots', 'videos', 'checksum_x', 'alternative_names', 'rating_count', 'total_rating_count', 'version_parent', 'hypes', 'url', 'franchises', 'follows', 'aggregated_rating_count', 'game_modes', 'player_perspectives', 'themes', 'game_engines', 'keywords', 'storyline', 'version_title', 'follows', 'bundles', 'franchise', 'checksum_y', 'second_id', 'offlinecoopmax', 'onlinecoopmax', 'onlinecoopmax'])
game_library.shape


(422434, 128)

In [88]:
game_library.head(2)

,id,category,created_at,name,slug,rating,dlcs,campaigncoop,dropin,lancoop,offlinecoop,offlinemax,onlinecoop,splitscreen,onlinemax,point-and-click,fighting,shooter,music,puzzle,racing,real-time-strategy-rts,role-playing-rpg,simulator,sport,strategy,turn-based-strategy-tbs,tactical,hack-and-slash-beat-em-up,quiz-trivia,pinball,adventure,indie,arcade,visual-novel,card-and-board-game,moba,Linux,Nintendo 64,Wii,PC (Microsoft Windows),PlayStation,PlayStation 2,PlayStation 3,Xbox,Xbox 360,DOS,Mac,Nintendo Entertainment System,Super Nintendo Entertainment System,Nintendo DS,Nintendo GameCube,Game Boy Color,Game Boy Advance,Sega Mega Drive/Genesis,Sega Saturn,Game Boy,Android,Nintendo 3DS,PlayStation Portable,iOS,Wii U,PlayStation Vita,Virtual Console (Nintendo),PlayStation 4,Xbox One,Family Computer Disk System,Arcade,Windows Phone,Apple II,Web browser,Odyssey,Commodore 16,New Nintendo 3DS,Thriller,Science fiction,Action,Horror,Survival,Fantasy,Historical,Stealth,Comedy,Business,Drama,Non-fiction,Kids,Sandbox,Open world,Warfare,Educational,Mystery,Party,Romance,Erotic,First person,Third person,Bird view / Isometric,Text,Side view,Virtual Reality,AuditoryDrama,single-player,multiplayer,co-operative,split-screen,massively-multiplayer-online-mmo,battle-royale,has_dlcs
0,40104,0,1498435200,Dogou Souken,dogou-souken,NaN,0,False,False,False,False,0.0,False,False,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,85031,0,1517392813,City Mysteries,city-mysteries,NaN,0,False,False,False,False,0.0,False,False,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [76]:
game_library = game_library.drop(columns=['remasters', 'summary', 'age_ratings', 'involved_companies', 'status', 'aggregated_rating', 'total_rating', 'remakes', 'ports', 'parent_game', 'aggregated_rating', 'multiplayer_modes', 'expansions', 'expanded_games', 'forks', 'standalone_expansions', 'second', 'version_description', 'title_version'])
                                        
                                          

In [87]:
game_library = game_library.drop(columns=['platform'])
game_library.shape

(422434, 109)

In [ ]:
df2['rating_bin'] = pd.cut(df2.rating, 
                           bins = [0,10,20, 30, 40, 50, 60, 70, 80, 90, 100],
                           labels = ['awful','very_bad','bad','unimpressive','average','fair','alright','good','great', 'subperb'])

### Dataframe with ratings

In [89]:
game_ratings = game_library[game_library['rating'].notnull()]
game_ratings.head()

,id,category,created_at,name,slug,rating,dlcs,campaigncoop,dropin,lancoop,offlinecoop,offlinemax,onlinecoop,splitscreen,onlinemax,point-and-click,fighting,shooter,music,puzzle,racing,real-time-strategy-rts,role-playing-rpg,simulator,sport,strategy,turn-based-strategy-tbs,tactical,hack-and-slash-beat-em-up,quiz-trivia,pinball,adventure,indie,arcade,visual-novel,card-and-board-game,moba,Linux,Nintendo 64,Wii,PC (Microsoft Windows),PlayStation,PlayStation 2,PlayStation 3,Xbox,Xbox 360,DOS,Mac,Nintendo Entertainment System,Super Nintendo Entertainment System,Nintendo DS,Nintendo GameCube,Game Boy Color,Game Boy Advance,Sega Mega Drive/Genesis,Sega Saturn,Game Boy,Android,Nintendo 3DS,PlayStation Portable,iOS,Wii U,PlayStation Vita,Virtual Console (Nintendo),PlayStation 4,Xbox One,Family Computer Disk System,Arcade,Windows Phone,Apple II,Web browser,Odyssey,Commodore 16,New Nintendo 3DS,Thriller,Science fiction,Action,Horror,Survival,Fantasy,Historical,Stealth,Comedy,Business,Drama,Non-fiction,Kids,Sandbox,Open world,Warfare,Educational,Mystery,Party,Romance,Erotic,First person,Third person,Bird view / Isometric,Text,Side view,Virtual Reality,AuditoryDrama,single-player,multiplayer,co-operative,split-screen,massively-multiplayer-online-mmo,battle-royale,has_dlcs
16,18248,0,1457458737,Beat City,beat-city,60.00000,0,False,False,False,False,0.0,False,False,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
23,100789,0,1525651200,Flying Arrow!.,flying-arrow--1,34.16185,0,False,False,False,False,0.0,False,False,0.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
42,75935,0,1510766425,Boinks,boinks,70.00000,0,False,False,False,False,0.0,False,False,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
47,88912,9,1519382760,Sonic the Hedgehog Classic,sonic-the-hedgehog-classic,80.00000,0,False,False,False,False,0.0,False,False,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
51,35350,0,1495832682,Vernon's Legacy,vernons-legacy,50.00000,0,False,False,False,False,0.0,False,False,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [90]:
game_ratings.shape

(158762, 109)

### Dataframe with nulls in rating column 

In [92]:
not_rated = game_library[game_library['rating'].isnull()]
not_rated.shape

(263672, 109)

### Splitting the dataframe

In [98]:
def split(game_ratings, stratify_by='rating'):
    # split df into train_validate 
    train_validate, test = train_test_split(game_ratings, test_size=.20, random_state=13)
    train, validate = train_test_split(train_validate, test_size=.3, random_state=13)

    X_train = train.drop(columns=['rating'])
    y_train = train[['rating']]

    X_validate = validate.drop(columns=['rating'])
    y_validate = validate[['rating']]

    X_test = test.drop(columns=['rating'])
    y_test = test[['rating']]

    return train, X_train, X_validate, X_test, y_train, y_validate, y_test

In [99]:
train, X_train, X_validate, X_test, y_train, y_validate, y_test = split(game_ratings, stratify_by='rating') #split data
train.head()

,id,category,created_at,name,slug,rating,dlcs,campaigncoop,dropin,lancoop,offlinecoop,offlinemax,onlinecoop,splitscreen,onlinemax,point-and-click,fighting,shooter,music,puzzle,racing,real-time-strategy-rts,role-playing-rpg,simulator,sport,strategy,turn-based-strategy-tbs,tactical,hack-and-slash-beat-em-up,quiz-trivia,pinball,adventure,indie,arcade,visual-novel,card-and-board-game,moba,Linux,Nintendo 64,Wii,PC (Microsoft Windows),PlayStation,PlayStation 2,PlayStation 3,Xbox,Xbox 360,DOS,Mac,Nintendo Entertainment System,Super Nintendo Entertainment System,Nintendo DS,Nintendo GameCube,Game Boy Color,Game Boy Advance,Sega Mega Drive/Genesis,Sega Saturn,Game Boy,Android,Nintendo 3DS,PlayStation Portable,iOS,Wii U,PlayStation Vita,Virtual Console (Nintendo),PlayStation 4,Xbox One,Family Computer Disk System,Arcade,Windows Phone,Apple II,Web browser,Odyssey,Commodore 16,New Nintendo 3DS,Thriller,Science fiction,Action,Horror,Survival,Fantasy,Historical,Stealth,Comedy,Business,Drama,Non-fiction,Kids,Sandbox,Open world,Warfare,Educational,Mystery,Party,Romance,Erotic,First person,Third person,Bird view / Isometric,Text,Side view,Virtual Reality,AuditoryDrama,single-player,multiplayer,co-operative,split-screen,massively-multiplayer-online-mmo,battle-royale,has_dlcs
300211,48274,11,1499960862,Teenage Mutant Ninja Turtles: Tournament Fighters,teenage-mutant-ninja-turtles-tournament-fighters,56.066018,0,False,False,False,False,2.0,False,False,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
244698,19914,0,1467882576,Charlie Murder,charlie-murder,69.028919,0,True,True,False,True,0.0,True,False,0.0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
126679,9384,0,1428090908,Kung Fu Strike: The Warrior's Rise,kung-fu-strike-the-warriors-rise,60.000000,0,True,False,False,True,0.0,False,False,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
394870,49120,0,1500011248,The Trail: Frontier Challenge,the-trail-frontier-challenge,60.000000,0,False,False,False,False,0.0,False,False,0.0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
9145,16253,0,1452846584,Alliance of Valiant Arms,alliance-of-valiant-arms,50.000000,0,False,True,False,False,0.0,True,False,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
